In [2]:
import pandas as pd
import numpy as np

Create a cell that contains two arrays , where the first integer represents the degree of the column, second integer the degree of rows, and the third the degree of diagonal. 

In [3]:
import sys

class Cell:
    def __init__(self):
        self.colour = 0  # 0: empty, 1: red, 2: black
        self.scores = {
            'vertical': [0, 0],
            'horizontal': [0, 0],
            'diagonal1': [0, 0],  # /
            'diagonal2': [0, 0]   # \
        }

    def reset(self):
        self.colour = 0
        for direction in self.scores:
            self.scores[direction] = [0, 0]

class Board:
    def __init__(self, rows: int = 6, cols: int = 7):
        self.rows = rows
        self.cols = cols
        self.cells = [[Cell() for _ in range(cols)] for _ in range(rows)]

    def reset(self):
        for row in self.cells:
            for cell in row:
                cell.reset()

    def is_valid_move(self, col: int) -> bool:
        return 0 <= col < self.cols and self.cells[0][col].colour == 0

    def drop_chip(self, col: int, colour: int) -> tuple[int, int]:
        if not self.is_valid_move(col):
            raise ValueError("Invalid move")

        for row in range(self.rows - 1, -1, -1):
            if self.cells[row][col].colour == 0:
                self.cells[row][col].colour = colour
                return row, col
            
    def update_vertical_score(self, row: int, col: int, colour: int) -> bool:
        idx = colour - 1
        count = 1
        # Check below the current cell
        for r in range(row + 1, min(row + 4, self.rows)):
            if self.cells[r][col].colour == colour:
                count += 1
            else:
                break
        return count >= 4

    def update_horizontal_score(self, row: int, col: int, colour: int) -> bool:
        idx = colour - 1
        count = 1
        # Check to the left
        for c in range(col - 1, max(col - 4, -1), -1):
            if self.cells[row][c].colour == colour:
                count += 1
            else:
                break
        # Check to the right
        for c in range(col + 1, min(col + 4, self.cols)):
            if self.cells[row][c].colour == colour:
                count += 1
            else:
                break
        return count >= 4

    def update_diagonal1_score(self, row: int, col: int, colour: int) -> bool:
        idx = colour - 1
        count = 1
        # Check top-left to bottom-right
        r, c = row - 1, col - 1
        while r >= 0 and c >= 0 and self.cells[r][c].colour == colour:
            count += 1
            r -= 1
            c -= 1
        r, c = row + 1, col + 1
        while r < self.rows and c < self.cols and self.cells[r][c].colour == colour:
            count += 1
            r += 1
            c += 1
        return count >= 4

    def update_diagonal2_score(self, row: int, col: int, colour: int) -> bool:
        idx = colour - 1
        count = 1
        # Check top-right to bottom-left
        r, c = row - 1, col + 1
        while r >= 0 and c < self.cols and self.cells[r][c].colour == colour:
            count += 1
            r -= 1
            c += 1
        r, c = row + 1, col - 1
        while r < self.rows and c >= 0 and self.cells[r][c].colour == colour:
            count += 1
            r += 1
            c -= 1
        return count >= 4

    def update_scores(self, row: int, col: int) -> bool:
        colour = self.cells[row][col].colour
        return (self.update_vertical_score(row, col, colour) or
                self.update_horizontal_score(row, col, colour) or
                self.update_diagonal1_score(row, col, colour) or
                self.update_diagonal2_score(row, col, colour))

class Game:
    def __init__(self, players: list[str], moves: list[str]):
        self.board = Board()
        self.players = players
        self.moves = moves

    def play(self) -> tuple[int, int]:
        for i, move in enumerate(self.moves):
            colour = 1 if i % 2 == 0 else 2
            col = int(move[1]) - 1
            
            row, col = self.board.drop_chip(col, colour)
            if self.board.update_scores(row, col):
                return colour, i + 1  # Return winner and move number
        
        return 0, len(self.moves)  # Draw

def parse_file(filename: str) -> list[Game]:
    games = []
    with open(filename, 'r') as file:
        game_data = []
        for line in file:
            line = line.strip()
            if line:
                game_data.append(line)
                if len(game_data) == 2:
                    players = game_data[0].split(', ')
                    moves = game_data[1].split(',')
                    games.append(Game(players, moves))
                    game_data = []
    return games

def process_games(games: list[Game]) -> list[str]:
    results = []
    
    for game in games:
        winner, winning_move = game.play()
        
        if winner == 0:
            result = f"Draw between {game.players[0]} and {game.players[1]}"
        else:
            winning_player = game.players[0] if winner == 1 else game.players[1]
            result = f"{winning_player} won in {winning_move} moves"
        
        results.append(result)
    return results

def main(filename: str):
    games = parse_file(filename)
    results = process_games(games)
    
    for result in results:
        print(result)

In [5]:
input_file = r'/Users/jvink/Personal/Random projects/devoteam/examples.txt'
main(input_file)

player_0 won in 9 moves
Draw between player_2 and player_3
player_4 won in 25 moves
